In [ ]:
%run ./check_DOVS_METHODS_OLD.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

# Analyze collected data

## AMI NonVee

In [ ]:
# base_dir_ami = r'C:\Users\s346557\Documents\LocalData\dovs_check\AMINonVee'
# base_dir_ede = r'C:\Users\s346557\Documents\LocalData\dovs_check\EndEvents'
# save_dir     = r'C:\Users\s346557\Documents\LocalData\dovs_check\Results'

# base_dir_ami = r'C:\Users\s346557\Documents\LocalData\dovs_check\forAmanda\SN_882735506\AMINonVee'
# base_dir_ede = r'C:\Users\s346557\Documents\LocalData\dovs_check\forAmanda\SN_882735506\EndEvents'
# save_dir     = r'C:\Users\s346557\Documents\LocalData\dovs_check\forAmanda\SN_882735506\Results'

# base_dir_ami = r'C:\Users\s346557\Documents\LocalData\dovs_check\ByOPCO\swp\AMINonVee'
# base_dir_ede = r'C:\Users\s346557\Documents\LocalData\dovs_check\ByOPCO\swp\EndEvents'
# save_dir     = r'C:\Users\s346557\Documents\LocalData\dovs_check\ByOPCO\swp\Results'

base_dir_ami = r'C:\Users\s346557\Documents\LocalData\dovs_check\20230806_20230812\AllOPCOs\AMINonVee'
base_dir_ede = r'C:\Users\s346557\Documents\LocalData\dovs_check\20230806_20230812\AllOPCOs\EndEvents'
save_dir     = r'C:\Users\s346557\Documents\LocalData\dovs_check\20230806_20230812\AllOPCOs\Results'

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
#-------------------------
paths = Utilities.find_all_paths(
    base_dir=base_dir_ami, 
    glob_pattern=r'ami_nonvee_[0-9]*.csv', 
    regex_pattern=None
)
paths=natsorted(paths)
#-------------------------
outg_rec_nbs_in_files = dict()
for path in paths:
    assert(path not in outg_rec_nbs_in_files.keys())
    df = GenAn.read_df_from_csv(path)
    outg_rec_nbs_in_files[path] = df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
outg_rec_nb_to_files_dict = invert_file_to_outg_rec_nbs_dict(outg_rec_nbs_in_files)
all_outg_rec_nbs = list(outg_rec_nb_to_files_dict.keys())

In [ ]:
#-------------------------
paths_ede = Utilities.find_all_paths(
    base_dir=base_dir_ede, 
    glob_pattern=r'end_events_[0-9]*.csv', 
    regex_pattern=None
)
paths_ede=natsorted(paths_ede)
#-------------------------
outg_rec_nbs_in_files_ede = dict()
for path in paths_ede:
    assert(path not in outg_rec_nbs_in_files_ede.keys())
    df = GenAn.read_df_from_csv(path)
    outg_rec_nbs_in_files_ede[path] = df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
outg_rec_nb_to_files_ede_dict = invert_file_to_outg_rec_nbs_dict(outg_rec_nbs_in_files_ede)
all_outg_rec_nbs_ede = list(outg_rec_nb_to_files_ede_dict.keys())

In [ ]:
#-------------------------
instvabc_slcr = DFSlicer(
    single_slicers = [
        dict(
            column='aep_derived_uom', 
            value='VOLT', 
            comparison_operator='=='
        ), 
        dict(
            column='aep_srvc_qlty_idntfr', 
            value=['INSTVA1', 'INSTVB1', 'INSTVC1'], 
            comparison_operator='isin'
        )
    ], 
    name='VOLT, INSTV(ABC)1', 
    join_single_slicers='and'
)
#-------------------------
volt_avg_slcr = DFSlicer(
    single_slicers = [
        dict(
            column='aep_derived_uom', 
            value='VOLT', 
            comparison_operator='=='
        ), 
        dict(
            column='aep_srvc_qlty_idntfr', 
            value='AVG', 
            comparison_operator='=='
        )
    ], 
    name='VOLT, AVG', 
    join_single_slicers='and'
)
#-------------------------
slicers=[instvabc_slcr, volt_avg_slcr]

In [ ]:
# start = time.time()
# outg_rec_nbs_to_remove = identify_outg_rec_nbs_to_remove(
#     paths=paths,
#     slicers=slicers, 
#     verbose=True
# )
# print(time.time()-start)

In [ ]:
# outg_rec_nbs_to_remove=[]

In [ ]:
include_suboutg_endpt_plots=True

calculate_by_PN = True
# combine_by_PN_likeness_thresh = pd.Timedelta('1 minutes')
combine_by_PN_likeness_thresh = pd.Timedelta('15 minutes')

expand_outg_search_time_tight = pd.Timedelta('1 hours')
expand_outg_search_time_loose = pd.Timedelta('12 hours')
use_est_outg_times=False
use_full_ede_outgs=False
run_outg_inclusion_assessment=True
max_pct_PNs_missing_allowed=0
# max_pct_PNs_missing_allowed=20
#-----
expand_outg_est_search_time = expand_outg_search_time_loose
if use_est_outg_times:
    expand_outg_search_time = expand_outg_search_time_tight
else:
    expand_outg_search_time = expand_outg_search_time_loose

In [ ]:
pdf_path = os.path.join(save_dir, r'Results.pdf')
pdf = PdfPages(pdf_path)
#-------------------------
pdf_path_dovs_beg = Utilities.append_to_path(
    pdf_path, 
    '_dovs_beg', 
    ext_to_find='.pdf', 
    append_to_end_if_ext_no_found=False
)
pdf_dovs_beg = PdfPages(pdf_path_dovs_beg)
#-------------------------
if include_suboutg_endpt_plots:
    pdf_path_2 = Utilities.append_to_path(
        pdf_path, 
        '_w_suboutg_endpt_plots', 
        ext_to_find='.pdf', 
        append_to_end_if_ext_no_found=False
    )
    pdf_2 = PdfPages(pdf_path_2)

In [ ]:
fig_num=0
all_detailed_summary_dfs=[]
all_detailed_summary_dfs_dovs_beg=[]
ci_cmi_summary_df = pd.DataFrame(columns=[
    'outg_rec_nb', 
    'ci_dovs',  'ci_ami', 'ci_ami_dovs_beg', 
    'cmi_dovs', 'cmi_ami', 'cmi_ami_dovs_beg'
])

#-------------------------
# Build dovs_df
dovs = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_std_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=all_outg_rec_nbs, 
        field_to_split='outg_rec_nbs', 
        include_premise=True
    ), 
    build_consolidated=True
)
dovs_df = dovs.df.copy()

#-------------------------
# Now, iterate through all outages
for i_outg, outg_rec_nb in enumerate(all_outg_rec_nbs):
    print(f'\n\ti_outg: {i_outg+1}/{len(all_outg_rec_nbs)}')
    print(f'\toutg_rec_nb = {outg_rec_nb}')
    #-----
    ami_df = GenAn.read_df_from_csv_batch(outg_rec_nb_to_files_dict[outg_rec_nb])
    #--------------------------------------------------
    ami_df_i = ami_df[ami_df['OUTG_REC_NB_GPD_FOR_SQL']==outg_rec_nb].copy()
    
    # Although I cannot yet call choose_best_slicer_and_perform_slicing and reduce_INSTV_ABC_1_vals_in_df, 
    #   as the standard cleaning and conversions must be done first, I am able to cut down the size of
    #   ami_df_i by joining the slicers with 'or' statements.
    # Thus, ami_df_i will be reduced to only the subset of data which will be considered in 
    #   choose_best_slicer_and_perform_slicing
    # As mentioned, this will cut down the size of ami_df_i and will also save time and resources by not having
    #   to run entire DF through cleaning and conversions procedures.
    ami_df_i = DFSlicer.combine_slicers_and_perform_slicing(
        df=ami_df_i, 
        slicers=slicers, 
        join_slicers='or'
    )
    if ami_df.shape[0]==0:
        continue
        
    #--------------------------------------------------
    ami_df_i = AMINonVee.perform_std_initiation_and_cleaning(ami_df_i)
    #-----
    # Should the following be added to AMINonVee.perform_std_initiation_and_cleaning?
    ami_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df_i, 
        time_col='starttimeperiod', 
        placement_col='starttimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    ami_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df_i, 
        time_col='endtimeperiod', 
        placement_col='endtimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    #--------------------------------------------------
    ami_df_i = choose_best_slicer_and_perform_slicing(
        df=ami_df_i, 
        slicers=slicers, 
        groupby_SN=True, 
        t_search_min_max=None, 
        time_col='starttimeperiod_local', 
        value_col=None, 
        SN_col='serialnumber', 
        return_sorted=True
    )

    ami_df_i = reduce_INSTV_ABC_1_vals_in_df(
        df=ami_df_i, 
        value_col='value', 
        aep_derived_uom_col='aep_derived_uom', 
        aep_srvc_qlty_idntfr_col='aep_srvc_qlty_idntfr', 
        output_aep_srvc_qlty_idntfr = 'INSTV(ABC)1'
    )

    if ami_df_i.shape[0]==0:
        continue
        
    #-------------------------
    # Each serial number should have a single value per time stamp
    assert(ami_df_i.groupby(['serialnumber', 'starttimeperiod_local']).ngroups == ami_df_i.shape[0])

    #-------------------------
    if run_outg_inclusion_assessment:
        to_include_i = assess_outage_inclusion_requirements(
            ami_df_i=ami_df_i, 
            outg_rec_nb=outg_rec_nb, 
            dovs_df=dovs_df, 
            max_pct_PNs_missing_allowed=max_pct_PNs_missing_allowed

        )
        if not to_include_i:
            print(f'outg_rec_nb={outg_rec_nb} did not pass inclusion requirements, skipping!!!!!')
            continue
    #-------------------------    
    n_SNs  = ami_df_i['serialnumber'].nunique()
    n_PNs  = ami_df_i['aep_premise_nb'].nunique()
    
    #----------------------------------------------------------------------------------------------------
    # NOTE: Can save time by grabbing ede_df_i then performing tz conversion and adding DOVS
    if outg_rec_nb not in outg_rec_nb_to_files_ede_dict.keys():
        ede_df_i=None
    else:
        ede_df = GenAn.read_df_from_csv_batch(outg_rec_nb_to_files_ede_dict[outg_rec_nb])
        ede_df_i = ede_df[ede_df['OUTG_REC_NB_GPD_FOR_SQL']==outg_rec_nb].copy()

        #-----
        ede_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
            df=ede_df_i, 
            time_col='valuesinterval', 
            placement_col='valuesinterval_local', 
            run_quick=True, 
            n_strip=6, 
            inplace=False
        )
        ede_df_i = AMIEndEvents.reduce_end_event_reasons_in_df(df=ede_df_i)
        #-----
        ede_cols_to_keep = [
            'valuesinterval_local', 
            'reason', 
            'serialnumber', 
            'aep_premise_nb', 
            'enddeviceeventtypeid', 
            'event_type', 
            'OUTG_REC_NB_GPD_FOR_SQL', 
            'trsf_pole_nb_GPD_FOR_SQL',
        ]
        ede_df_i = ede_df_i[ede_cols_to_keep]
        
    #----------------------------------------------------------------------------------------------------
    dovs_df_i = DOVSOutages.retrieve_outage_from_dovs_df(
        dovs_df=dovs_df, 
        outg_rec_nb=outg_rec_nb, 
        outg_rec_nb_idfr='index', 
        assert_outg_rec_nb_found=True
    )
    assert(dovs_df_i.shape[0]==1)
    # Get the outage time from DOVS
    dovs_outg_t_beg_end = dovs_df_i.iloc[0][['DT_OFF_TS_FULL', 'DT_ON_TS']].tolist()
    assert(len(dovs_outg_t_beg_end)==2)
    dovs_outg_t_beg, dovs_outg_t_end = dovs_outg_t_beg_end
    #-------------------------
    # Get the CI and CMI from DOVS
    ci_cmi_dovs = dovs_df_i.iloc[0][['CI_NB', 'CMI_NB']].tolist()
    assert(len(ci_cmi_dovs)==2)
    ci_dovs, cmi_dovs = ci_cmi_dovs
    #-------------------------
    # Get the number of premises from DOVS
    n_PNs_dovs = len(set(dovs_df_i.iloc[0]['premise_nbs']))
    #-------------------------
    # Get the outage number from DOVS
    outage_nb = dovs_df_i.iloc[0]['OUTAGE_NB']
    
    #--------------------------------------------------
    res_dict = calculate_ci_cmi_w_ami_w_ede_help(
        df=ami_df_i, 
        ede_df=ede_df_i, 
        dovs_outg_t_beg_end=dovs_outg_t_beg_end, 
        expand_outg_search_time=expand_outg_search_time, 
        conservative_estimate=True, 
        est_ede_kwargs=dict(use_full_ede_outgs=use_full_ede_outgs), 
        audit_selection_method='ede only', 
        return_CI_SNs=False, 
        use_est_outg_times=use_est_outg_times, 
        pct_SNs_required_for_outage_est=0, 
        expand_outg_est_search_time=expand_outg_est_search_time, 
        use_only_overall_endpoints_of_est_outg_times=False, 
        time_col='starttimeperiod_local', 
        value_col='value', 
        SN_col='serialnumber', 
        return_all_best_ests=True, 
        return_all_best_ests_type='pd.DataFrame'
    )
    #-----
    ci_ami       = res_dict['CI']
    cmi_ami      = res_dict['CMI']
    best_ests_df = res_dict['all_best_ests']
    #--------------------------------------------------
    if calculate_by_PN and best_ests_df.shape[0]>0:
        try:
            best_ests_df = combine_PNs_in_best_ests_df(
                best_ests_df, 
                likeness_thresh = combine_by_PN_likeness_thresh, 
                SN_col = 'SN', 
                PN_col = 'PN', 
                i_outg_col = 'i_outg'     
            )
            ci_ami  = best_ests_df['PN'].nunique()
            cmi_ami = (best_ests_df['winner_max']-best_ests_df['winner_min']).sum().total_seconds()/60
        except:
            print(f'outg_rec_nb={outg_rec_nb} failed combine_PNs_in_best_ests_df, so skipping')
            continue
    #--------------------------------------------------
    if best_ests_df.shape[0]>0:
        best_ests_df_dovs_beg = alter_best_ests_df_using_dovs_outg_t_beg(
            best_ests_df=best_ests_df,
            dovs_df=dovs_df_i, 
            outg_rec_nb=outg_rec_nb
        )
        if calculate_by_PN:
            ci_ami_dovs_beg  = best_ests_df_dovs_beg['PN'].nunique()
        else:
            ci_ami_dovs_beg  = best_ests_df_dovs_beg['SN'].nunique()
        cmi_ami_dovs_beg = (best_ests_df_dovs_beg['winner_max']-best_ests_df_dovs_beg['winner_min']).sum().total_seconds()/60
    else:
        best_ests_df_dovs_beg = best_ests_df.copy()
        ci_ami_dovs_beg = ci_ami
        cmi_ami_dovs_beg = cmi_ami        
    #--------------------------------------------------
    if best_ests_df.shape[0]>0:
        means_df, best_ests_df_w_db_lbl = get_mean_times_w_dbscan(
            best_ests_df, 
            eps_min=5, 
            min_samples=2, 
            ests_to_include_in_clustering=['winner_min', 'winner_max'],
            ests_to_include_in_output=[
                'winner_min', 'winner_max', 
                'conservative_min', 'conservative_max', 
                'zero_times_min', 'zero_times_max'
            ], 
            return_labelled_best_ests_df=True
        )
        #-----
        detailed_summary_df_i = build_detailed_summary_df(
            means_df=means_df, 
            best_ests_df_w_db_lbl=best_ests_df_w_db_lbl,
            CI_tot=ci_ami, 
            CMI_tot=cmi_ami, 
            n_PNs_ami=n_PNs,
            outg_rec_nb=outg_rec_nb, 
            dovs_df_i=dovs_df_i, 
            db_label_col='db_label', 
            winner_min_col='winner_min', 
            winner_max_col='winner_max', 
            PN_col='PN' if calculate_by_PN else 'SN', 
            i_outg_col='i_outg'
        )
        all_detailed_summary_dfs.append(detailed_summary_df_i)
    else:
        means_df, best_ests_df_w_db_lbl = None, None
    #--------------------------------------------------
    if best_ests_df_dovs_beg.shape[0]>0:
        means_df_dovs_beg, best_ests_df_dovs_beg_w_db_lbl = get_mean_times_w_dbscan(
            best_ests_df_dovs_beg, 
            eps_min=5, 
            min_samples=2, 
            ests_to_include_in_clustering=['winner_min', 'winner_max'],
            ests_to_include_in_output=[
                'winner_min', 'winner_max', 
                'conservative_min', 'conservative_max', 
                'zero_times_min', 'zero_times_max'
            ], 
            return_labelled_best_ests_df=True
        )
        #-----
        detailed_summary_df_dovs_beg_i = build_detailed_summary_df(
            means_df=means_df_dovs_beg, 
            best_ests_df_w_db_lbl=best_ests_df_dovs_beg_w_db_lbl,
            CI_tot=ci_ami_dovs_beg, 
            CMI_tot=cmi_ami_dovs_beg, 
            n_PNs_ami=n_PNs,
            outg_rec_nb=outg_rec_nb, 
            dovs_df_i=dovs_df_i, 
            db_label_col='db_label', 
            winner_min_col='winner_min', 
            winner_max_col='winner_max', 
            PN_col='PN' if calculate_by_PN else 'SN', 
            i_outg_col='i_outg'
        )
        all_detailed_summary_dfs_dovs_beg.append(detailed_summary_df_dovs_beg_i)
    else:
        means_df_dovs_beg, best_ests_df_dovs_beg_w_db_lbl = None, None
    #-------------------------
    ci_cmi_summary_df = pd.concat([
        ci_cmi_summary_df, 
        pd.DataFrame(
            dict(
                outg_rec_nb=outg_rec_nb, 
                ci_dovs=ci_dovs,   ci_ami=ci_ami, ci_ami_dovs_beg=ci_ami_dovs_beg, 
                cmi_dovs=cmi_dovs, cmi_ami=cmi_ami, cmi_ami_dovs_beg=cmi_ami_dovs_beg
            ), 
            index=[ci_cmi_summary_df.shape[0]]
        )
    ])
    #----------------------------------------------------------------------------------------------------
    # ######################### PLOTTING #########################
    #----------------------------------------------------------------------------------------------------
    if means_df is not None:
        cnsrvtv_out_t_beg = np.min([means_df.min().min(), dovs_outg_t_beg])
        cnsrvtv_out_t_end = np.max([means_df.max().max(), dovs_outg_t_end])
    else:
        cnsrvtv_out_t_beg = dovs_outg_t_beg
        cnsrvtv_out_t_end = dovs_outg_t_end
    #--------------------------------------------------
    # Get SNs which were completely out, not out at all, or partially out during
    #   the outage (as defined by the DOVS outage times, NOT the estimated times above)
    pct_time_out_required_for_outage=0
    dct_full_part_not = get_full_part_not_outage_subset_dfs(
        df=ami_df_i, 
        out_t_beg=cnsrvtv_out_t_beg-pd.Timedelta('12 hours'),
        out_t_end=cnsrvtv_out_t_end+pd.Timedelta('12 hours'), 
        pct_time_out_required_for_outage=pct_time_out_required_for_outage, 
        return_SNs=True, 
        exclusive_partials=False, 
        time_col='starttimeperiod_local', 
        value_col='value', 
        SN_col='serialnumber'    
    )
    #----------
    outg_SNs = dct_full_part_not['full_SNs']
    ami_df_i_out = dct_full_part_not['full_df']
    #-----
    not_outg_SNs = dct_full_part_not['not_SNs']
    ami_df_i_not_out = dct_full_part_not['not_df']
    #-----
    part_outg_SNs = dct_full_part_not['part_SNs']
    ami_df_i_part_out = dct_full_part_not['part_df']        

    #----------------------------------------------------------------------------------------------------
    fig, axs = plot_all_out_not(
        fig_num              = fig_num, 
        ami_df_i             = ami_df_i, 
        ami_df_i_part_out    = ami_df_i_part_out, 
        ami_df_i_not_out     = ami_df_i_not_out, 
        dovs_outg_t_beg      = dovs_outg_t_beg, 
        dovs_outg_t_end      = dovs_outg_t_end, 
        cnsrvtv_out_t_beg    = cnsrvtv_out_t_beg, 
        cnsrvtv_out_t_end    = cnsrvtv_out_t_end, 
        means_df             = means_df, 
        outg_rec_nb          = outg_rec_nb, 
        outage_nb            = outage_nb, 
        n_PNs_dovs           = n_PNs_dovs, 
        n_PNs                = n_PNs, 
        n_SNs                = n_SNs, 
        ci_dovs              = ci_dovs, 
        cmi_dovs             = cmi_dovs, 
        ci_ami               = ci_ami, 
        cmi_ami              = cmi_ami, 
        ci_ami_dovs_beg      = ci_ami_dovs_beg, 
        cmi_ami_dovs_beg     = cmi_ami_dovs_beg, 
        expand_time          = pd.Timedelta('1 hour'), 
        mean_keys_to_include = ['winner', 'conservative', 'zero_times']
    )

    fig_num += 1
    pdf.savefig(fig, bbox_inches='tight')
    if include_suboutg_endpt_plots:
        pdf_2.savefig(fig, bbox_inches='tight')
    plt.close(fig)
    
    #----------------------------------------------------------------------------------------------------
    fig, axs = plot_all_out_not(
        fig_num              = fig_num, 
        ami_df_i             = ami_df_i, 
        ami_df_i_part_out    = ami_df_i_part_out, 
        ami_df_i_not_out     = ami_df_i_not_out, 
        dovs_outg_t_beg      = dovs_outg_t_beg, 
        dovs_outg_t_end      = dovs_outg_t_end, 
        cnsrvtv_out_t_beg    = cnsrvtv_out_t_beg, 
        cnsrvtv_out_t_end    = cnsrvtv_out_t_end, 
        means_df             = means_df_dovs_beg, 
        outg_rec_nb          = outg_rec_nb, 
        outage_nb            = outage_nb, 
        n_PNs_dovs           = n_PNs_dovs, 
        n_PNs                = n_PNs, 
        n_SNs                = n_SNs, 
        ci_dovs              = ci_dovs, 
        cmi_dovs             = cmi_dovs, 
        ci_ami               = ci_ami, 
        cmi_ami              = cmi_ami, 
        ci_ami_dovs_beg      = ci_ami_dovs_beg, 
        cmi_ami_dovs_beg     = cmi_ami_dovs_beg, 
        expand_time          = pd.Timedelta('1 hour'), 
        mean_keys_to_include = ['winner']
    )

    fig_num += 1
    pdf_dovs_beg.savefig(fig, bbox_inches='tight')
    plt.close(fig)
    
    #----------------------------------------------------------------------------------------------------
    if means_df is not None and include_suboutg_endpt_plots:
        fig, axs = plot_suboutg_endpts(
            fig_num               = fig_num, 
            ami_df_i              = ami_df_i, 
            means_df              = means_df, 
            best_ests_df_w_db_lbl = best_ests_df_w_db_lbl, 
            dovs_outg_t_beg       = dovs_outg_t_beg, 
            dovs_outg_t_end       = dovs_outg_t_end, 
            outg_rec_nb           = outg_rec_nb, 
            expand_time           = pd.Timedelta('15 minutes'), 
            mean_keys_to_include  = ['winner', 'conservative', 'zero_times']
        )
        #-------------------------
        fig_num += 1
        pdf_2.savefig(fig, bbox_inches='tight')     
        plt.close(fig)
#----------------------------------------------------------------------------------------------------
detailed_summary_df          = pd.concat(all_detailed_summary_dfs)
detailed_summary_df_dovs_beg = pd.concat(all_detailed_summary_dfs_dovs_beg)

In [ ]:
pdf.close()
pdf_dovs_beg.close()
if include_suboutg_endpt_plots:
    pdf_2.close()
detailed_summary_df.to_pickle(os.path.join(save_dir, r'detailed_summary.pkl'))
detailed_summary_df_dovs_beg.to_pickle(os.path.join(save_dir, r'detailed_summary_dovs_beg.pkl'))

In [ ]:
ci_cmi_summary_df['ci_dovs']         = ci_cmi_summary_df['ci_dovs'].astype(float)
ci_cmi_summary_df['ci_ami']          = ci_cmi_summary_df['ci_ami'].astype(float)
ci_cmi_summary_df['ci_ami_dovs_beg'] = ci_cmi_summary_df['ci_ami_dovs_beg'].astype(float)
#-----
ci_cmi_summary_df['delta_ci_dovs_ami']  = ci_cmi_summary_df['ci_dovs']-ci_cmi_summary_df['ci_ami']
ci_cmi_summary_df['delta_cmi_dovs_ami'] = ci_cmi_summary_df['cmi_dovs']-ci_cmi_summary_df['cmi_ami']
#-----
ci_cmi_summary_df['delta_ci_dovs_ami_dovs_beg']  = ci_cmi_summary_df['ci_dovs']-ci_cmi_summary_df['ci_ami_dovs_beg']
ci_cmi_summary_df['delta_cmi_dovs_ami_dovs_beg'] = ci_cmi_summary_df['cmi_dovs']-ci_cmi_summary_df['cmi_ami_dovs_beg']
#-----
# For plotting purposes, make a outg_rec_in column which is simply 0 to delta_df.shape[0]-1
ci_cmi_summary_df['outg_rec_int'] = range(ci_cmi_summary_df.shape[0])
#-----
ci_cmi_summary_df.to_pickle(os.path.join(save_dir, r'ci_cmi_summary.pkl'))

In [ ]:
assert(0)

In [ ]:
ci_cmi_summary_df

# ===========================================================

In [ ]:
def build_summary_df_subset_slicer(
    col,
    slicer=None, 
    min_val=None,
    max_val=None    
):
    r"""
    Purpose of this function is essentially to make it easier to input a list of
    columns to get_summary_df_subset
    """
    #-------------------------
    if slicer is None:
        slicer = DFSlicer()
    else:
        assert(isinstance(slicer, DFSlicer))
    #-------------------------
    if min_val is None and max_val is None:
        return slicer
    #-------------------------
    if min_val:
        slicer.add_single_slicer(        
            dict(
                column=col, 
                value=min_val, 
                comparison_operator='>='
            )
        )
    if max_val:
        slicer.add_single_slicer(        
            dict(
                column=col, 
                value=max_val, 
                comparison_operator='<='
            )
        )
    #-------------------------
    return slicer


def get_summary_df_subset(
    df,
    cols,
    min_val=None,
    max_val=None,
    label_int_col=None
):
    r"""
    cols should either be a single column in df or a list of columns in df
    """
    #-------------------------
    if min_val is None and max_val is None:
        return df
    #-------------------------
    if not isinstance(cols, list):
        assert(cols in df.columns.tolist())
        cols = [cols]
    #-----
    for col in cols:
        assert(col in df.columns.tolist())
    #-------------------------
    slicer = DFSlicer()
    for col in cols:
        slicer = build_summary_df_subset_slicer(
            col=col,
            slicer=slicer, 
            min_val=min_val,
            max_val=max_val    
        )
    #-------------------------
    return_df = slicer.perform_slicing(df.copy())
    #-------------------------
    if label_int_col is not None:
        return_df[label_int_col] = range(return_df.shape[0])
    #-------------------------
    return return_df

In [ ]:
def make_summary_df_cols_pretty(
    df,
    rename_cols=None
):
    r"""
    If rename_cols is None, use the standard rename_cols, std_rename_cols
    """
    #-------------------------
    std_rename_cols = {
        'ci_dovs' : 'CI DOVS',
        'ci_ami' : 'CI AMI',
        'ci_ami_dovs_beg' : 'CI AMI w/ DOVS Beg.', 
        #-----
        'cmi_dovs' : 'CMI DOVS',
        'cmi_ami' : 'CMI AMI',
        'ci_ami_dovs_beg' : 'CMI AMI w/ DOVS Beg.', 
        #-----
        'delta_ci_dovs_ami' : 'Delta CI DOVS vs AMI', 
        'delta_ci_dovs_ami_dovs_beg' : 'Delta CI DOVS vs AMI w/ DOVS Beg.', 
        #-----
        'delta_cmi_dovs_ami' : 'Delta CMI DOVS vs AMI', 
        'delta_cmi_dovs_ami_dovs_beg' : 'Delta CMI DOVS vs AMI w/ DOVS Beg.',
    }    
    #-------------------------
    if rename_cols is None:
        rename_cols = std_rename_cols
    #-------------------------
    rename_cols = {k:v for k,v in rename_cols.items() if k in df.columns.tolist()}
    df = df.rename(columns=rename_cols)
    return df

In [ ]:
def identify_delta_cols_in_summary_df(df):
    r"""
    Basically, just return columns containing 'delta' (case insensitive).  
    """
    #-------------------------
    delta_cols = [x for x in df.columns.tolist() if 'delta' in x.lower()]
    return delta_cols

def get_delta_cols_from_summary_df(df):
    r"""
    """
    #-------------------------
    return df[identify_delta_cols_in_summary_df(df)]

In [ ]:
def identify_cmi_cols_in_summary_df(df):
    r"""
    Basically, just return columns containing 'cmi' (case insensitive).  
    """
    #-------------------------
    cmi_cols = [x for x in df.columns.tolist() if 'cmi' in x.lower()]
    return cmi_cols

def get_cmi_cols_from_summary_df(df):
    r"""
    """
    #-------------------------
    return df[identify_cmi_cols_in_summary_df(df)]

In [ ]:
def identify_ci_cols_in_summary_df(df):
    r"""
    Basically, just return columns containing 'ci' (case insensitive).  
    """
    #-------------------------
    ci_cols = [x for x in df.columns.tolist() if 'ci' in x.lower()]
    return ci_cols

def get_ci_cols_from_summary_df(df):
    r"""
    """
    #-------------------------
    return df[identify_ci_cols_in_summary_df(df)]

In [ ]:
def build_final_summary_results(
    summary_df, 
    limits_coll, 
    calc_col, 
    dovs_col, 
    subset_col, 
    metric, 
    are_limits_pct, 
    assert_cols_seem_approp=True
):
    r"""
    e.g.:
            final_df_cmi = build_final_summary_results(
                summary_df=summary_df, 
                limits_coll = [
                    [None, None], 
                    [-10000, 10000], 
                    [-1000, 1000], 
                    [-100, 100], 
                    [-10, 10], 
                ], 
                calc_col = 'Delta CMI DOVS vs AMI', 
                dovs_col = 'CMI DOVS', 
                subset_col = None, 
                metric = 'CMI', 
                are_limits_pct = False
            )
    """
    #-------------------------
    assert(metric.upper() in ['CI', 'CMI'])
    metric=metric.upper()
    #-------------------------
    if subset_col is None:
        subset_col = calc_col
    #-------------------------
    if metric not in calc_col.upper():
        print(f"Warning: metric={metric} not found in calc_col={calc_col}")
        if assert_cols_seem_approp:
            assert(0)
    #-----
    if metric not in dovs_col.upper():
        print(f"Warning: metric={metric} not found in dovs_col={dovs_col}")
        if assert_cols_seem_approp:
            assert(0)    
    #-----
    if metric not in subset_col.upper():
        print(f"Warning: metric={metric} not found in subset_col={subset_col}")
        if assert_cols_seem_approp:
            assert(0)    
    #-------------------------
    min_delta_col = 'Min Delta'
    max_delta_col = 'Max Delta'
    delta_col = f'Delta {metric}'
    if are_limits_pct:
        min_delta_col = 'Min Delta (%)'
        max_delta_col = 'Max Delta (%)'
    #-----
    return_df = pd.DataFrame(columns=[
        min_delta_col, 
        max_delta_col, 
        dovs_col,  
        delta_col
    ])
    #-------------------------
    for i,limits_i in enumerate(limits_coll):
        summary_df_i = get_summary_df_subset(
            df=summary_df,
            cols=subset_col,
            min_val=limits_i[0],
            max_val=limits_i[1]
        )
        #------------------------------------
        return_df = pd.concat([
            return_df, 
            pd.DataFrame(
                {
                    min_delta_col: f"{limits_i[0]}", 
                    max_delta_col: f"{limits_i[1]}", 
                    dovs_col: f"{summary_df_i[dovs_col].sum()}",  
                    delta_col: f"{np.round(summary_df_i[calc_col].sum(), decimals=2)} ({np.round(100*summary_df_i[calc_col].sum()/summary_df_i[dovs_col].sum(), decimals=2)}%)",  
                }, 
                index=[return_df.shape[0]]
            )
        ])
    #-------------------------
    return return_df

In [ ]:
def replace_delta_with_greek(
    txt, 
    case_insensitive=False
):
    r"""
    For plotting purposes, this function will replace 'Delta' with '$\Delta$' (and 'delta' with '$\delta$').
    If case_insensitive==True, any Delta ('Delta', 'DELta', 'delta', etc.) will be replaced with '$\Delta$'
    """
    if not case_insensitive:
        txt = txt.replace('Delta', '$\Delta$')
        txt = txt.replace('delta', '$\delta$')
    else:
        # Cannot simply replace in-place in a while loop as I am replacing 'Delta', with '$\Delta$', so the 
        #   while loop will be infinite!!!!
        # Therefore, first find all occurrenced of 'Delta', then replace
        found_itr = re.finditer(r'Delta', txt, flags=re.IGNORECASE)
        #-----
        found_idxs=[]
        for found in found_itr:
            found_idxs.append([found.start(), found.end()])
        #-----
        # NOTE: Must replace from the right, so the indices of those to-be-replaced remain correct after
        #       others are replaced
        found_idxs = natsorted(found_idxs, reverse=True)
        #-----
        if len(found_idxs)>0:
            for found_idxs_i in found_idxs:
                txt_0 = txt[:found_idxs_i[0]]
                txt_1 = '$\Delta$'
                txt_2 = txt[found_idxs_i[1]:]
                #-----
                txt=txt_0+txt_1+txt_2
    return txt

In [ ]:
def draw_delta_vs_outg_rec_int(
    ax, 
    summary_df, 
    limits, 
    calc_col, 
    dovs_col, 
    subset_col, 
    metric, 
    are_limits_pct, 
    outg_rec_int_col='outg_rec_int', 
    assert_cols_seem_approp=True
):
    r"""
    """
    #-------------------------
    assert(metric.upper() in ['CI', 'CMI'])
    metric=metric.upper()
    #-------------------------
    if subset_col is None:
        subset_col = calc_col
    #-------------------------
    if metric not in calc_col.upper():
        print(f"Warning: metric={metric} not found in calc_col={calc_col}")
        if assert_cols_seem_approp:
            assert(0)
    #-----
    if metric not in dovs_col.upper():
        print(f"Warning: metric={metric} not found in dovs_col={dovs_col}")
        if assert_cols_seem_approp:
            assert(0)    
    #-----
    if metric not in subset_col.upper():
        print(f"Warning: metric={metric} not found in subset_col={subset_col}")
        if assert_cols_seem_approp:
            assert(0)
    #--------------------------------------------------
    summary_df_i = get_summary_df_subset(
        df=summary_df,
        cols=subset_col,
        min_val=limits[0],
        max_val=limits[1], 
        label_int_col=outg_rec_int_col
    )
    sns.lineplot(ax=ax, x=outg_rec_int_col, y=calc_col, data=summary_df_i)
    if are_limits_pct:
        ax.set_title(f"Min/Max $\Delta$ (%) = [{limits[0]}, {limits[1]}]", fontsize=20)
    else:
        ax.set_title(f"Min/Max $\Delta$ = [{limits[0]}, {limits[1]}]", fontsize=20)
    #-------------------------
    Plot_General.set_general_plotting_args(
        ax=ax, 
        tick_args =[
            dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
            dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
        ], 
        xlabel_args=dict(xlabel='Outage', fontsize=16), 
        ylabel_args=dict(ylabel=replace_delta_with_greek(ax.get_ylabel()), fontsize=16)
    )


    if are_limits_pct:
        ax.text(1.05, 0.70, f"Min/Max $\Delta$ (%) = [{limits[0]}, {limits[1]}]", ha='left', va='center', transform=ax.transAxes, fontsize='xx-large', fontdict={'family':'monospace'})
    else:
        ax.text(1.05, 0.70, f"Min/Max $\Delta$ = [{limits[0]}, {limits[1]}]", ha='left', va='center', transform=ax.transAxes, fontsize='xx-large', fontdict={'family':'monospace'})
    ax.text(1.05, 0.50, f"Sums", ha='left', va='center', transform=ax.transAxes, fontsize='xx-large', fontdict={'family':'monospace'})
    ax.text(1.05, 0.40, f"{dovs_col} = {np.round(summary_df_i[dovs_col].sum(), decimals=2)}", ha='left', va='center', transform=ax.transAxes, fontsize='xx-large', fontdict={'family':'monospace'})
    ax.text(1.05, 0.30, f"$\Delta$AMI = {np.round(summary_df_i[calc_col].sum(), decimals=2)} ({np.round(100*summary_df_i[calc_col].sum()/summary_df_i[dovs_col].sum(), decimals=2)}%)", 
            ha='left', va='center', transform=ax.transAxes, fontsize='xx-large', fontdict={'family':'monospace'})

In [ ]:
summary_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\ByOPCO\oh\Results\ci_cmi_summary.pkl')
# summary_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\oh\Results\ci_cmi_summary.pkl')

In [ ]:
summary_df

In [ ]:
identify_ci_cols_in_summary_df(summary_df)

In [ ]:
summary_df.iloc[summary_df.reset_index()['delta_cmi_dovs_ami'].idxmin()]

In [ ]:
get_delta_cols_from_summary_df(summary_df).sum()

In [ ]:
get_delta_cols_from_summary_df(summary_df).describe()

In [ ]:
fig,ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='outg_rec_int', y='delta_cmi_dovs_ami', data=summary_df)

In [ ]:
fig,ax = Plot_General.default_subplots()
Plot_Hist.plot_hist(
    ax=ax, 
    df=summary_df, 
    x_col='delta_cmi_dovs_ami', 
    min_max_and_bin_size=200
)

In [ ]:
fig,ax = Plot_General.default_subplots()
Plot_Hist.plot_hist(
    ax=ax, 
    df=summary_df, 
    x_col='delta_cmi_dovs_ami', 
    min_max_and_bin_size=[-200, 1000, 50], 
    include_over_underflow=True
)

In [ ]:
summary_df.describe()

In [ ]:
summary_df = make_summary_df_cols_pretty(summary_df)

In [ ]:
get_delta_cols_from_summary_df(summary_df).sum()

In [ ]:
get_delta_cols_from_summary_df(summary_df).describe()

In [ ]:
fig_save_dir = r'C:\Users\s346557\Documents\Presentations\DOVS_check\Figures\forAmanda'

In [ ]:
fig,ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='outg_rec_int', y='Delta CMI DOVS vs AMI', data=summary_df)
Plot_General.set_general_plotting_args(
    ax,
    title_args=dict(label='$\Delta$CMI: DOVS vs AMI', fontsize=18),
    xlabel_args=dict(xlabel='Outage', fontsize=14), 
    ylabel_args=dict(ylabel='$\Delta$CMI', fontsize=14)
)
# Plot_General.save_fig(
#     fig=fig, 
#     save_dir=fig_save_dir, 
#     save_name='DeltaCMI_vs_Outage_AMI.png', 
#     bbox_inches='tight'
# )

In [ ]:
fig,ax = Plot_General.default_subplots()
Plot_Hist.plot_hist(
    ax=ax, 
    df=summary_df, 
    x_col='Delta CMI DOVS vs AMI', 
    min_max_and_bin_size=200
)

In [ ]:
fig,ax = Plot_General.default_subplots()
Plot_Hist.plot_hist(
    ax=ax, 
    df=summary_df, 
    x_col='Delta CMI DOVS vs AMI', 
    min_max_and_bin_size=[-1000, 1000, 25], 
    include_over_underflow=True
)
Plot_General.set_general_plotting_args(
    ax,
    title_args=dict(label='$\Delta$CMI: DOVS vs AMI', fontsize=18),
    xlabel_args=dict(xlabel='$\Delta$CMI', fontsize=14), 
    ylabel_args=dict(ylabel='Counts', fontsize=14)
)

# Plot_General.save_fig(
#     fig=fig, 
#     save_dir=fig_save_dir, 
#     save_name='DeltaCMI_DOVS_vs_AMI.png', 
#     bbox_inches='tight'
# )

In [ ]:
fig,ax = Plot_General.default_subplots()
Plot_Hist.plot_hist(
    ax=ax, 
    df=summary_df, 
    x_col='Delta CI DOVS vs AMI', 
    min_max_and_bin_size=200
)

In [ ]:
fig,ax = Plot_General.default_subplots()
Plot_Hist.plot_hist(
    ax=ax, 
    df=summary_df, 
    x_col='Delta CI DOVS vs AMI', 
    min_max_and_bin_size=[-3, 10, 1], 
    include_over_underflow=True
)
Plot_General.set_general_plotting_args(
    ax,
    title_args=dict(label='$\Delta$CI: DOVS vs AMI', fontsize=18),
    xlabel_args=dict(xlabel='$\Delta$CI', fontsize=14), 
    ylabel_args=dict(ylabel='Counts', fontsize=14)
)

# Plot_General.save_fig(
#     fig=fig, 
#     save_dir=fig_save_dir, 
#     save_name='DeltaCI_DOVS_vs_AMI.png', 
#     bbox_inches='tight'
# )

In [ ]:
summary_df['CI DOVS'].sum()

In [ ]:
summary_df['CMI DOVS'].sum()

In [ ]:
print('----- summary_df -----')
print(f'shape[0] = {summary_df.shape[0]}')
print(summary_df[['CI DOVS', 'CMI DOVS']].sum())
print()
#-------------------------

In [ ]:
summary_df['Pct. Delta CI DOVS vs AMI']  = 100.*summary_df['Delta CI DOVS vs AMI']/summary_df['CI DOVS']
summary_df['Pct. Delta CMI DOVS vs AMI'] = 100.*summary_df['Delta CMI DOVS vs AMI']/summary_df['CMI DOVS']
#-----
summary_df['Pct. Delta CI DOVS vs AMI w/ DOVS Beg.']  = 100.*summary_df['Delta CI DOVS vs AMI w/ DOVS Beg.']/summary_df['CI DOVS']
summary_df['Pct. Delta CMI DOVS vs AMI w/ DOVS Beg.'] = 100.*summary_df['Delta CMI DOVS vs AMI w/ DOVS Beg.']/summary_df['CMI DOVS']

In [ ]:
fig, ax = Plot_General.default_subplots(n_x=1, n_y=1)
sns.lineplot(ax=ax, x='outg_rec_int', y='Pct. Delta CMI DOVS vs AMI', data=summary_df)

In [ ]:
fig, ax = Plot_General.default_subplots(n_x=1, n_y=1)
sns.lineplot(ax=ax, x='outg_rec_int', y='Pct. Delta CMI DOVS vs AMI', data=summary_df)
ax.set_ylim([-1000, 150])

In [ ]:
fig, ax = Plot_General.default_subplots(n_x=1, n_y=1)
sns.lineplot(ax=ax, x='outg_rec_int', y='Pct. Delta CI DOVS vs AMI', data=summary_df)

In [ ]:
fig, ax = Plot_General.default_subplots(n_x=1, n_y=1)
sns.lineplot(ax=ax, x='outg_rec_int', y='Delta CMI DOVS vs AMI', data=summary_df)

In [ ]:
final_df_cmi = build_final_summary_results(
    summary_df=summary_df, 
    limits_coll = [
        [None, None], 
        [-10000, 10000], 
        [-1000, 1000], 
        [-100, 100], 
        [-10, 10], 
    ], 
    calc_col = 'Delta CMI DOVS vs AMI', 
    dovs_col = 'CMI DOVS', 
    subset_col = None, 
    metric = 'CMI', 
    are_limits_pct = False
)

In [ ]:
final_df_cmi_pct = build_final_summary_results(
    summary_df=summary_df, 
    limits_coll = [
        [None, None], 
        [-100, 100], 
        [-50, 50], 
        [-20, 20], 
        [-10, 10], 
    ], 
    calc_col = 'Delta CMI DOVS vs AMI', 
    dovs_col = 'CMI DOVS', 
    subset_col = 'Pct. Delta CMI DOVS vs AMI', 
    metric = 'CMI', 
    are_limits_pct = True
)

In [ ]:
final_df_ci = build_final_summary_results(
    summary_df=summary_df, 
    limits_coll = [
        [None, None], 
        [-50, 50], 
        [-25, 25], 
        [-10, 10], 
        [-5, 5], 
    ], 
    calc_col = 'Delta CI DOVS vs AMI', 
    dovs_col = 'CI DOVS', 
    subset_col = None, 
    metric = 'CI', 
    are_limits_pct = False
)

In [ ]:
final_df_ci_pct = build_final_summary_results(
    summary_df=summary_df, 
    limits_coll = [
        [None, None], 
        [-100, 100], 
        [-50, 50], 
        [-20, 20], 
        [-10, 10], 
    ], 
    calc_col = 'Delta CI DOVS vs AMI', 
    dovs_col = 'CI DOVS', 
    subset_col = 'Pct. Delta CI DOVS vs AMI', 
    metric = 'CI', 
    are_limits_pct = True
)

In [ ]:
# pdf = PdfPages(r'C:\Users\s346557\Documents\LocalData\dovs_check\Results\SummaryPages_v2.pdf')

In [ ]:
# limits_coll = [
#     [None, None], 
#     [-100, 100], 
#     [-50, 50], 
#     [-20, 20], 
#     [-10, 10],
# ]
limits_coll = [
    [None, None], 
    [-100, 100], 
    [-20, 20]
]

calc_col = 'Delta CMI DOVS vs AMI'
dovs_col = 'CMI DOVS'
subset_col = 'Pct. Delta CMI DOVS vs AMI'
metric='CMI'
are_limits_pct=True

#--------------------------------------------------
final_df = build_final_summary_results(
    summary_df=summary_df, 
    limits_coll = limits_coll, 
    calc_col = calc_col, 
    dovs_col = dovs_col, 
    subset_col = subset_col, 
    metric = metric, 
    are_limits_pct = are_limits_pct
)

#--------------------------------------------------
fig, axs = Plot_General.default_subplots(n_x=1, n_y=len(limits_coll)+1)
Plot_General.adjust_subplots_args(fig, Plot_General.get_subplots_adjust_args(hspace=0.30))
#-------------------------
axs[0].axis('off')
tbl = pd.plotting.table(
    axs[0], 
    final_df, 
    cellLoc = 'center', 
    rowLoc = 'center', 
    bbox=[0,0,1,1], 
    colColours=['white', 'white', 'white', 'green']
)
tbl.set_fontsize(16)
#-------------------------
for i,ci_limits in enumerate(limits_coll):
    draw_delta_vs_outg_rec_int(
        ax=axs[i+1], 
        summary_df=summary_df, 
        limits=ci_limits, 
        calc_col=calc_col, 
        dovs_col=dovs_col, 
        subset_col=subset_col, 
        metric=metric, 
        are_limits_pct=are_limits_pct
    )
#-------------------------
if are_limits_pct:
    fig.suptitle('Subsets extracted using relative $\Delta$s', y=0.90, fontsize='xx-large')
else:
    fig.suptitle('Subsets extracted using raw $\Delta$s', y=0.90, fontsize='xx-large')
    
# pdf.savefig(fig, bbox_inches='tight')

In [ ]:
# limits_coll = [
#     [None, None], 
#     [-100, 100], 
#     [-50, 50], 
#     [-20, 20], 
#     [-10, 10],
# ]
limits_coll = [
    [None, None], 
    [-100, 100], 
    [-20, 20]
]

calc_col = 'Delta CMI DOVS vs AMI w/ DOVS Beg.'
dovs_col = 'CMI DOVS'
subset_col = 'Pct. Delta CMI DOVS vs AMI w/ DOVS Beg.'
metric='CMI'
are_limits_pct=True

#--------------------------------------------------
final_df = build_final_summary_results(
    summary_df=summary_df, 
    limits_coll = limits_coll, 
    calc_col = calc_col, 
    dovs_col = dovs_col, 
    subset_col = subset_col, 
    metric = metric, 
    are_limits_pct = are_limits_pct
)

#--------------------------------------------------
fig, axs = Plot_General.default_subplots(n_x=1, n_y=len(limits_coll)+1)
Plot_General.adjust_subplots_args(fig, Plot_General.get_subplots_adjust_args(hspace=0.30))
#-------------------------
axs[0].axis('off')
tbl = pd.plotting.table(
    axs[0], 
    final_df, 
    cellLoc = 'center', 
    rowLoc = 'center', 
    bbox=[0,0,1,1], 
    colColours=['white', 'white', 'white', 'green']
)
tbl.set_fontsize(16)
#-------------------------
for i,ci_limits in enumerate(limits_coll):
    draw_delta_vs_outg_rec_int(
        ax=axs[i+1], 
        summary_df=summary_df, 
        limits=ci_limits, 
        calc_col=calc_col, 
        dovs_col=dovs_col, 
        subset_col=subset_col, 
        metric=metric, 
        are_limits_pct=are_limits_pct
    )
#-------------------------
if are_limits_pct:
    fig.suptitle('Subsets extracted using relative $\Delta$s', y=0.90, fontsize='xx-large')
else:
    fig.suptitle('Subsets extracted using raw $\Delta$s', y=0.90, fontsize='xx-large')
    
# pdf.savefig(fig, bbox_inches='tight')

In [ ]:
# limits_coll = [
#     [None, None], 
#     [-10000, 10000], 
#     [-1000, 1000], 
#     [-100, 100], 
#     [-10, 10], 
# ]

limits_coll = [
    [None, None], 
    [-10000, 10000], 
    [-100, 100]
]

calc_col = 'Delta CMI DOVS vs AMI'
dovs_col='CMI DOVS'
subset_col=None
metric='CMI'
are_limits_pct=False

#--------------------------------------------------
final_df = build_final_summary_results(
    summary_df=summary_df, 
    limits_coll = limits_coll, 
    calc_col = calc_col, 
    dovs_col = dovs_col, 
    subset_col = subset_col, 
    metric = metric, 
    are_limits_pct = are_limits_pct
)

#--------------------------------------------------
fig, axs = Plot_General.default_subplots(n_x=1, n_y=len(limits_coll)+1)
Plot_General.adjust_subplots_args(fig, Plot_General.get_subplots_adjust_args(hspace=0.30))
#-------------------------
axs[0].axis('off')
tbl = pd.plotting.table(
    axs[0], 
    final_df, 
    cellLoc = 'center', 
    rowLoc = 'center', 
    bbox=[0,0,1,1], 
    colColours=['white', 'white', 'white', 'green']
)
tbl.set_fontsize(16)
#-------------------------
for i,ci_limits in enumerate(limits_coll):
    draw_delta_vs_outg_rec_int(
        ax=axs[i+1], 
        summary_df=summary_df, 
        limits=ci_limits, 
        calc_col=calc_col, 
        dovs_col=dovs_col, 
        subset_col=subset_col, 
        metric=metric, 
        are_limits_pct=are_limits_pct
    )
#-------------------------
if are_limits_pct:
    fig.suptitle('Subsets extracted using relative $\Delta$s', y=0.90, fontsize='xx-large')
else:
    fig.suptitle('Subsets extracted using raw $\Delta$s', y=0.90, fontsize='xx-large')
    
# pdf.savefig(fig, bbox_inches='tight')

In [ ]:
# limits_coll = [
#     [None, None], 
#     [-10000, 10000], 
#     [-1000, 1000], 
#     [-100, 100], 
#     [-10, 10], 
# ]

limits_coll = [
    [None, None], 
    [-10000, 10000], 
    [-100, 100]
]

calc_col = 'Delta CMI DOVS vs AMI w/ DOVS Beg.'
dovs_col='CMI DOVS'
subset_col=None
metric='CMI'
are_limits_pct=False

#--------------------------------------------------
final_df = build_final_summary_results(
    summary_df=summary_df, 
    limits_coll = limits_coll, 
    calc_col = calc_col, 
    dovs_col = dovs_col, 
    subset_col = subset_col, 
    metric = metric, 
    are_limits_pct = are_limits_pct
)

#--------------------------------------------------
fig, axs = Plot_General.default_subplots(n_x=1, n_y=len(limits_coll)+1)
Plot_General.adjust_subplots_args(fig, Plot_General.get_subplots_adjust_args(hspace=0.30))
#-------------------------
axs[0].axis('off')
tbl = pd.plotting.table(
    axs[0], 
    final_df, 
    cellLoc = 'center', 
    rowLoc = 'center', 
    bbox=[0,0,1,1], 
    colColours=['white', 'white', 'white', 'green']
)
tbl.set_fontsize(16)
#-------------------------
for i,ci_limits in enumerate(limits_coll):
    draw_delta_vs_outg_rec_int(
        ax=axs[i+1], 
        summary_df=summary_df, 
        limits=ci_limits, 
        calc_col=calc_col, 
        dovs_col=dovs_col, 
        subset_col=subset_col, 
        metric=metric, 
        are_limits_pct=are_limits_pct
    )
#-------------------------
if are_limits_pct:
    fig.suptitle('Subsets extracted using relative $\Delta$s', y=0.90, fontsize='xx-large')
else:
    fig.suptitle('Subsets extracted using raw $\Delta$s', y=0.90, fontsize='xx-large')
    
# pdf.savefig(fig, bbox_inches='tight')

In [ ]:
# limits_coll = [
#     [None, None], 
#     [-100, 100], 
#     [-50, 50], 
#     [-20, 20], 
#     [-10, 10],
# ]

limits_coll = [
    [None, None], 
    [-100, 100], 
    [-20, 20]
]

calc_col = 'Delta CI DOVS vs AMI'
dovs_col = 'CI DOVS'
subset_col = 'Pct. Delta CI DOVS vs AMI'
metric='CI'
are_limits_pct=True

#--------------------------------------------------
final_df = build_final_summary_results(
    summary_df=summary_df, 
    limits_coll = limits_coll, 
    calc_col = calc_col, 
    dovs_col = dovs_col, 
    subset_col = subset_col, 
    metric = metric, 
    are_limits_pct = are_limits_pct
)

#--------------------------------------------------
fig, axs = Plot_General.default_subplots(n_x=1, n_y=len(limits_coll)+1)
Plot_General.adjust_subplots_args(fig, Plot_General.get_subplots_adjust_args(hspace=0.30))
#-------------------------
axs[0].axis('off')
tbl = pd.plotting.table(
    axs[0], 
    final_df, 
    cellLoc = 'center', 
    rowLoc = 'center', 
    bbox=[0,0,1,1], 
    colColours=['white', 'white', 'white', 'green']
)
tbl.set_fontsize(16)
#-------------------------
for i,ci_limits in enumerate(limits_coll):
    draw_delta_vs_outg_rec_int(
        ax=axs[i+1], 
        summary_df=summary_df, 
        limits=ci_limits, 
        calc_col=calc_col, 
        dovs_col=dovs_col, 
        subset_col=subset_col, 
        metric=metric, 
        are_limits_pct=are_limits_pct
    )
#-------------------------
if are_limits_pct:
    fig.suptitle('Subsets extracted using relative $\Delta$s', y=0.90, fontsize='xx-large')
else:
    fig.suptitle('Subsets extracted using raw $\Delta$s', y=0.90, fontsize='xx-large')
    
# pdf.savefig(fig, bbox_inches='tight')

In [ ]:
# limits_coll = [
#     [None, None], 
#     [-50, 50], 
#     [-25, 25], 
#     [-10, 10], 
#     [-5, 5], 
# ]

limits_coll = [
    [None, None], 
    [-50, 50], 
    [-10, 10]
]


calc_col = 'Delta CI DOVS vs AMI'
dovs_col='CI DOVS'
subset_col=None
metric='CI'
are_limits_pct=False

#--------------------------------------------------
final_df = build_final_summary_results(
    summary_df=summary_df, 
    limits_coll = limits_coll, 
    calc_col = calc_col, 
    dovs_col = dovs_col, 
    subset_col = subset_col, 
    metric = metric, 
    are_limits_pct = are_limits_pct
)

#--------------------------------------------------
fig, axs = Plot_General.default_subplots(n_x=1, n_y=len(limits_coll)+1)
Plot_General.adjust_subplots_args(fig, Plot_General.get_subplots_adjust_args(hspace=0.30))
#-------------------------
axs[0].axis('off')
tbl = pd.plotting.table(
    axs[0], 
    final_df, 
    cellLoc = 'center', 
    rowLoc = 'center', 
    bbox=[0,0,1,1], 
    colColours=['white', 'white', 'white', 'green']
)
tbl.set_fontsize(16)
#-------------------------
for i,ci_limits in enumerate(limits_coll):
    draw_delta_vs_outg_rec_int(
        ax=axs[i+1], 
        summary_df=summary_df, 
        limits=ci_limits, 
        calc_col=calc_col, 
        dovs_col=dovs_col, 
        subset_col=subset_col, 
        metric=metric, 
        are_limits_pct=are_limits_pct
    )
#-------------------------
if are_limits_pct:
    fig.suptitle('Subsets extracted using relative $\Delta$s', y=0.90, fontsize='xx-large')
else:
    fig.suptitle('Subsets extracted using raw $\Delta$s', y=0.90, fontsize='xx-large')
    
# pdf.savefig(fig, bbox_inches='tight')

In [ ]:
# pdf.close()

In [ ]:
import PyPDF2
import pdfrw
import pagelabels

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!
# NOTE: Last time I ran, PyPDF2.PdfWriter messed up text in images, but
#       pypdf.PdfMerger seemed to work well

#--------------------------------------------------
# First, write merged PDF file
pdfs = [
    r'C:\Users\s346557\Documents\LocalData\dovs_check\Results\FinalDeliverableContents.pdf', 
    r'C:\Users\s346557\Documents\LocalData\dovs_check\Results\SummaryPages.pdf', 
    r'C:\Users\s346557\Documents\LocalData\dovs_check\Results\Results_ede_only.pdf'
]
merger = PyPDF2.PdfWriter()
for pdf in pdfs:
    merger.append(pdf)

merger.write(r'C:\Users\s346557\Documents\LocalData\dovs_check\Results\DOVSCorrection_forAmanda.pdf')
merger.close()
#--------------------------------------------------
# Next, change the pages from FinalDeliverableContents to be numbered with Roman numerals
n_pages_contents_desc = len(PyPDF2.PdfReader(r'C:\Users\s346557\Documents\LocalData\dovs_check\Results\FinalDeliverableContents.pdf').pages)
#-----
roman_labels = pagelabels.PageLabelScheme(
    startpage=0, # the index of the page of the PDF where the labels will start
    style="roman lowercase", # See options in PageLabelScheme.styles()
    prefix="",
    firstpagenum=1 # number to attribute to the first page of this index
)
reg_labels = pagelabels.PageLabelScheme(
    startpage=n_pages_contents_desc, 
    style="arabic", 
    prefix="",
    firstpagenum=1 
)
#-----
reader = PdfReader(r'C:\Users\s346557\Documents\LocalData\dovs_check\Results\DOVSCorrection_forAmanda.pdf')
labels = pagelabels.PageLabels.from_pdf(reader)
labels.append(roman_labels) 
labels.append(reg_labels)
#-----
labels.write(reader)
writer = pdfrw.PdfWriter()
writer.trailer = reader
writer.write(r'C:\Users\s346557\Documents\LocalData\dovs_check\Results\DOVSCorrection_forAmanda.pdf')

In [ ]:
limits_coll = [
    [None, None], 
    [-100, 100], 
    [-50, 50], 
    [-20, 20], 
    [-10, 10], 
]

calc_col = 'Delta CI DOVS vs AMI'
dovs_col='CI DOVS'
subset_col = 'Pct. Delta CI DOVS vs AMI'
metric='CI'
are_limits_pct=True


fig, axs = Plot_General.default_subplots(n_x=1, n_y=len(limits_coll))
Plot_General.adjust_subplots_args(fig, Plot_General.get_subplots_adjust_args(hspace=0.30))

for i,ci_limits in enumerate(limits_coll):
    draw_delta_vs_outg_rec_int(
        ax=axs[i], 
        summary_df=summary_df, 
        limits=ci_limits, 
        calc_col=calc_col, 
        dovs_col=dovs_col, 
        subset_col=subset_col, 
        metric=metric, 
        are_limits_pct=are_limits_pct
    )

In [ ]:
limits_coll = [
    [None, None], 
    [-10000, 10000], 
    [-1000, 1000], 
    [-100, 100], 
    [-10, 10], 
]

calc_col = 'Delta CMI DOVS vs AMI'
dovs_col='CMI DOVS'
subset_col=None
metric='CMI'
are_limits_pct=False


fig, axs = Plot_General.default_subplots(n_x=1, n_y=len(limits_coll))
Plot_General.adjust_subplots_args(fig, Plot_General.get_subplots_adjust_args(hspace=0.30))

for i,ci_limits in enumerate(limits_coll):
    draw_delta_vs_outg_rec_int(
        ax=axs[i], 
        summary_df=summary_df, 
        limits=ci_limits, 
        calc_col=calc_col, 
        dovs_col=dovs_col, 
        subset_col=subset_col, 
        metric=metric, 
        are_limits_pct=are_limits_pct
    )
    


In [ ]:
limits_coll = [
    [None, None], 
    [-100, 100], 
    [-50, 50], 
    [-20, 20], 
    [-10, 10], 
]

calc_col = 'Delta CMI DOVS vs AMI'
dovs_col='CMI DOVS'
subset_col = 'Pct. Delta CMI DOVS vs AMI'
metric='CMI'
are_limits_pct=True


fig, axs = Plot_General.default_subplots(n_x=1, n_y=len(limits_coll))
Plot_General.adjust_subplots_args(fig, Plot_General.get_subplots_adjust_args(hspace=0.30))

for i,ci_limits in enumerate(limits_coll):
    draw_delta_vs_outg_rec_int(
        ax=axs[i], 
        summary_df=summary_df, 
        limits=ci_limits, 
        calc_col=calc_col, 
        dovs_col=dovs_col, 
        subset_col=subset_col, 
        metric=metric, 
        are_limits_pct=are_limits_pct
    )
    


In [ ]:
fig,ax = Plot_General.default_subplots()
Plot_Hist.plot_hist(
    ax=ax, 
    df=summary_df, 
    x_col='Delta CMI DOVS vs AMI', 
    min_max_and_bin_size=200
)

In [ ]:
fig,ax = Plot_General.default_subplots()
Plot_Hist.plot_hist(
    ax=ax, 
    df=summary_df, 
    x_col='Delta CMI DOVS vs AMI', 
    min_max_and_bin_size=[-500, 1000, 50], 
    include_over_underflow=True
)
Plot_General.set_general_plotting_args(
    ax,
    title_args=dict(label='$\Delta$CMI: DOVS vs AMI', fontsize=18),
    xlabel_args=dict(xlabel='$\Delta$CMI', fontsize=14), 
    ylabel_args=dict(ylabel='Counts', fontsize=14)
)

# Plot_General.save_fig(
#     fig=fig, 
#     save_dir=fig_save_dir, 
#     save_name='DeltaCMI_DOVS_vs_AMI.png', 
#     bbox_inches='tight'
# )

In [ ]:
fig,ax = Plot_General.default_subplots()
Plot_Hist.plot_hist(
    ax=ax, 
    df=summary_df, 
    x_col='Delta CI DOVS vs AMI', 
    min_max_and_bin_size=200
)

In [ ]:
fig,ax = Plot_General.default_subplots()
Plot_Hist.plot_hist(
    ax=ax, 
    df=summary_df, 
    x_col='Delta CI DOVS vs AMI', 
    min_max_and_bin_size=[-10, 25, 1], 
    include_over_underflow=True
)
Plot_General.set_general_plotting_args(
    ax,
    title_args=dict(label='$\Delta$CI: DOVS vs AMI', fontsize=18),
    xlabel_args=dict(xlabel='$\Delta$CI', fontsize=14), 
    ylabel_args=dict(ylabel='Counts', fontsize=14)
)

# Plot_General.save_fig(
#     fig=fig, 
#     save_dir=fig_save_dir, 
#     save_name='DeltaCI_DOVS_vs_AMI.png', 
#     bbox_inches='tight'
# )

In [ ]:
df1 = detailed_summary_df.copy()
df2 = detailed_summary_df_NEW.copy().reset_index().set_index(['OUTAGE_NB', 'OUTG_REC_NB', 'Outage Subset'])
df2=df2.rename(columns={'n_PNs_DOVS':'n_PNs_dovs'})

In [ ]:
df1.equals(df2[df1.columns])

In [ ]:
df1

In [ ]:
df2[df1.columns]

In [ ]:
df2

In [ ]:
detailed_summary_df_NEW